In [1]:
#For more info about this code extract, look at lab10_code_extraction.ipynb
from glob import glob

In [2]:
#A list of notebooks
hw12_folders = glob('../notebooks/**/hw12.ipynb', recursive=True)

In [3]:
#Number of notebooks that have saved history
len(hw12_folders)

13

In [4]:
import json

In [5]:
#Create a list of dictionaries. Each dictionary is the metadata for one of the 160 notebooks
hw12s = []
for file in hw12_folders: 
    with open(file, 'r') as f: 
        hw12s.append(json.load(f))


In [6]:
#Num files with history
len(hw12s)

13

In [7]:
#Example metadata of each cell for the 4th notebook
hw12s[3]['cells']

[{'cell_type': 'markdown',
  'metadata': {},
  'source': ["# 'Homework' 12: Conditional Probability and General Review\n",
   '\n',
   "This notebook won't be graded!  It contains exercises designed to help you review for the final exam.  To simulate exam conditions, consider printing it out and completing it in pencil."]},
 {'cell_type': 'markdown',
  'metadata': {},
  'source': ['Run the cell below to prepare the notebook.']},
 {'cell_type': 'code',
  'execution_count': 1,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': ["# Run this cell to set up the notebook, but please don't change it.\n",
   'import numpy as np\n',
   'from datascience import *\n',
   '\n',
   '# These lines do some fancy plotting magic.\n',
   'import matplotlib\n',
   '%matplotlib inline\n',
   'import matplotlib.pyplot as plt\n',
   "plt.style.use('fivethirtyeight')\n",
   'import warnings\n',
   "warnings.simplefilter('ignore', FutureWarning)\n",
   'from matplotlib import patches\n',
   'fro

In [8]:
#Num cells in the 4th notebook
len(hw12s[3]['cells'])

74

In [9]:
def getCodeInCell(cell):
    """
    CELL - The metadata for one cell. 
    return - A list of code in string format.
            The number of elements in this list equals
            the number of times CELL was ran. 
            
            The number of elements in each inner list equals
            the number of lines in CELL.
            
            If the outer list doesn't contain list but just
            strings, then the number of lines of code in CELL was 1.
    """
    
    allCode = []
    
    # If CELL is a code cell and metadata isn't collapsed 
    # (there was no output from CELL), then look into metadata 
    # and extract from "metadata -> history" of CELL.
    if isCodeCell(cell) \
    and not cell['metadata']['collapsed'] \
    and 'history' in cell['metadata'] \
    and len(cell['metadata']['history']) >= 1:
        for past_code in cell['metadata']['history']:
            code = re.split(r'\n', past_code['code'])
            allCode.append(code)
                
    # If CELL is a markdown cell, extract from "source" of CELL.
    elif isMarkDownCell(cell):
        for answer_line in cell['source']:
            if answer_line == '\n':
                continue
            code = re.split(r'\n', answer_line)
            if code[len(code) - 1] == '':
                code = code[:len(code) - 1]
            allCode.append(code)
    
    # If metadata is collapsed, then extract from "source" of CELL.
    else:   
        allCode.extend(cell['source'])
    return allCode
        

In [10]:
#Returns whether CELL is a code cell.
def isCodeCell(cell):
    return cell['cell_type'] == 'code'

#Returns whether CELL is a markdown cell.
def isMarkDownCell(cell):
    return cell['cell_type'] == 'markdown'

In [11]:
import re


In [12]:
#example code cell
hw12s[3]['cells'][15]

{'cell_type': 'markdown',
 'metadata': {'for_assignment_type': 'student'},
 'source': ['**Null hypothesis:** Drinking diet drinks does not influence BMI change. Any BMI change is not due to drinking. \n',
  '\n',
  '**Alternative hypothesis:** Drinking diet drinks influences BMI change.\n',
  '\n',
  'no test is needed because we can build a confidence interval for BMI change. If 0 is in the interval then null is true. If 0 is not in the interval, then drinking is influential and we will have an estimate of BMI change.']}

In [13]:
#testing extracting answer from markdown cell
getCodeInCell(hw12s[3]['cells'][15])

[['**Null hypothesis:** Drinking diet drinks does not influence BMI change. Any BMI change is not due to drinking. '],
 ['**Alternative hypothesis:** Drinking diet drinks influences BMI change.'],
 ['no test is needed because we can build a confidence interval for BMI change. If 0 is in the interval then null is true. If 0 is not in the interval, then drinking is influential and we will have an estimate of BMI change.']]

In [14]:
#The regex pattern that matches the string that would 
#denote the start of a question. Unfortunately, there are 2 types of questions.
Q_PATTERN1 = re.compile("#### Question (\\d+)\\\n")
Q_PATTERN2 = re.compile("\\*\\*Question (\\d+):\\*\\*.*")

In [15]:
#The regex patterns that match titles of the 6 sections of this homework.
SECTION1_PATTERN = re.compile("## 1\\. Do Diet Drinks Cause Weight Gain\?\\\n")
SECTION2_PATTERN = re.compile("## 2\\. How to Avoid Road Rage with Bayes Rule\\\n")
SECTION3_PATTERN = re.compile("## 3\\. Marginal Histograms\\\n")
SECTION4_PATTERN = re.compile("## 4\\. A Potpourri of Tests\\\n")
SECTION5_PATTERN = re.compile("## 5\\. Review of Prediction Intervals\\\n")
SECTION6_PATTERN = re.compile("## 6\\. Detecting Spam\\\n")

#returns the section number of CELL, or False if none exists
def check_section(cell):
    s = cell['source'][0]
    if SECTION1_PATTERN.match(s):
        return 1
    elif SECTION2_PATTERN.match(s):
        return 2
    elif SECTION3_PATTERN.match(s):
        return 3
    elif SECTION4_PATTERN.match(s):
        return 4
    elif SECTION5_PATTERN.match(s):
        return 5
    elif SECTION6_PATTERN.match(s):
        return 6
    else: 
        return False
   

In [16]:
SECTION4_PATTERN.match(hw12s[0]['cells'][38]['source'][0])

<_sre.SRE_Match object; span=(0, 27), match='## 4. A Potpourri of Tests\n'>

In [17]:
Q_PATTERN1.match(hw12s[0]['cells'][39]['source'][0])

<_sre.SRE_Match object; span=(0, 16), match='#### Question 1\n'>

In [18]:
# An class that supports functionality to look at 
# the current head of the list. 
# Also supports the hasNext method from Java iterators.
class CellsIterator:
    def __init__(self, L):
        self.L = L
        self.i = 0
        self.n = len(L)
    def __iter__(self):
        return self
    def next(self):
        if self.i < self.n:
            result = self.L[self.i]
            self.i += 1
            return result
        else:
            raise StopIteration()
    def current(self):
        return self.L[self.i]
    def hasNext(self):
        return self.i < self.n - 1
    __next__ = next

In [19]:
# Process extracting the answer for question CURR_Q.
# Returns the answer as well as an indicator of whether we processed
# the last question for the current section we're in. 
def process_q(cells_iterator, curr_q):
    section, current = re.split("\.", curr_q)
    section = int(section)
    current = int(current)
    if section == 1:
        if current in [1, 2, 4]: 
            return (False, process_True(cells_iterator))
        else:
            return (current == 6, process_1(cells_iterator))
    elif section == 2:
        return (current == 4, process_1(cells_iterator))
    elif section == 3: 
        return (True, process_False(cells_iterator))
    elif section == 4: 
        return (current == 3,  process_1(cells_iterator))
    elif section == 5: 
        if current in [1, 2, 3]:
            return (False, process_True(cells_iterator))
        else: 
            return (True, process_1(cells_iterator))
    elif section == 6:
        return (current == 4, process_1(cells_iterator))
    else: 
        return "wot"
   

In [20]:
# Get code in the nearest cell that is a markdown cell. 
# Intended for questions whose answers are in words. However,
# sometimes students add a code cell to display information 
# before writing their word answer.
def process_1(cells_iterator):
    while not isMarkDownCell(cells_iterator.current()):
        cells_iterator.next()
    return getCodeInCell(cells_iterator.next())

# Get code in cells in iterator until hitting a markdown cell.
def process_True(cells_iterator):
    result = []
    while not isMarkDownCell(cells_iterator.current()):
        result.append(getCodeInCell(cells_iterator.next()))
    return result

# Get code in 2nd and 3rd cells in iterator (used only for question 3.1)
def process_False(cells_iterator):
    result = []
    cells_iterator.next()
    result.append(getCodeInCell(cells_iterator.current()))
    cells_iterator.next()
    result.append(getCodeInCell(cells_iterator.current()))
    return result
    

In [21]:
CI = iter(CellsIterator(hw12s[3]['cells']))
for i in range(39):
    CI.next()
CI.current()
process_False(CI)

[[['**Histogram A:** ![](var1.png)'], ['**Histogram B:** ![](var2.png)']],
 [['**Histogram for Line 1:**'], ['**Explanation:**']]]

In [22]:
#A tiny bit of testing
CI = iter(CellsIterator(hw12s[3]['cells']))
for i in range(14):
    CI.next()
process_1(CI)

[['**Null hypothesis:** Drinking diet drinks does not influence BMI change. Any BMI change is not due to drinking. '],
 ['**Alternative hypothesis:** Drinking diet drinks influences BMI change.'],
 ['no test is needed because we can build a confidence interval for BMI change. If 0 is in the interval then null is true. If 0 is not in the interval, then drinking is influential and we will have an estimate of BMI change.']]

In [23]:
#A bit more testing
CI = iter(CellsIterator(hw12s[1]['cells']))
while not SECTION1_PATTERN.match(CI.next()['source'][0]):
    _ = 0
CI.next()
CI.next()
CI.next()
process_True(CI)


[[['def drink(value):',
   '    if value == 0:',
   "        return 'False'",
   '    else:',
   "        return 'True'",
   '',
   'drinks = make_array()',
   'N = len(diet.column(1))',
   'for i in np.arange(N):',
   '    consumes = drink(diet.column(1).item(i))',
   '    drinks = np.append(drinks, consumes)',
   '    ',
   'drinks'],
  ['def drink(value):',
   '    if value == 0:',
   "        return 'False'",
   '    else:',
   "        return 'True'",
   '',
   'drinks = make_array()',
   'N = len(diet.column(1))',
   'for i in np.arange(N):',
   '    consumes = drink(diet.column(1).item(i))',
   '    drinks = np.append(drinks, consumes)',
   '    ',
   'drinks']],
 [["drink_or_not = diet.with_columns('Drink', drinks )", 'drink_or_not'],
  ["drink_or_not = diet.with_columns('Drink', drinks )", 'drink_or_not'],
  ["drink_or_not = diet.with_columns('Drink', drinks )", 'drink_or_not']]]

In [24]:
def extract_code_from_student(student_num):
    ## NOTE: Assumes that all code that pertains to a particular coding question
    # is contained in the code cells that immediately follow the markdown
    # cell that states the question.
    
    qs = {}
    curr_section = False
    curr_q = False
    check = False
    cells = CellsIterator(hw12s[student_num]['cells'])
    
    while cells.hasNext():
        curr = cells.next()
        
        # If a section hasn't been found yet, check CURR for a section number.
        if not curr_section:
            curr_section = check_section(curr) 
        else: 
            
            # If a section number has been found but not a question number, 
            # check CURR for a question number.
            if isMarkDownCell(curr) and not curr_q:
                for s in curr['source']:
                    if Q_PATTERN1.match(s):
                        qNum = Q_PATTERN1.search(s).group(1)
                        qs[str(curr_section) + "." + qNum] = []
                        curr_q = str(curr_section) + "." + qNum
                        break
                    elif Q_PATTERN2.match(s):
                        qNum = Q_PATTERN2.search(s).group(1)
                        qs[str(curr_section) + "." + qNum] = []
                        curr_q = str(curr_section) + "." + qNum
                        break
                        
            # If a question number has been found, extract answer.
            # If we've processed a question that is at the end of the
            # current section, then set CURR_SECTION to False so that
            # we start looking for a section number again.
            if curr_q:
                finished_section, code = process_q(cells, curr_q)
                qs[curr_q] = code
                curr_q = False
                if finished_section:
                    curr_section = False
    return qs
                
            
        

In [25]:
questions = {"1.1", "1.2", "1.3", "1.4", "1.5", "1.6",
             "2.1", "2.2", "2.3", "2.4", 
             "3.1", 
             "4.1", "4.2", "4.3",
             "5.1", "5.2", "5.3", "5.4", 
             "6.1", "6.2", "6.3", "6.4"}

In [26]:
[questions - extract_code_from_student(i).keys() for i in range(13)]

[set(),
 set(),
 {'3.1'},
 set(),
 {'2.3', '3.1'},
 set(),
 set(),
 set(),
 set(),
 {'2.3', '3.1'},
 set(),
 set(),
 set()]

In [27]:
correct = [0, 1, 3, 5, 6, 7, 8, 10, 11, 12]

In [28]:
hw12s_questions = {q:{i:extract_code_from_student(i)[q] for i in correct} for q in questions}

In [29]:
# A dict of dicts. Keys in outer dict are question numbers that map to another inner dict.
# Keys of inner dict are indices (that denote a student) that map to that student's answer.
hw12s_questions

{'1.1': {0: [[["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not'],
    ["drink_or_not = diet.with_column('Drink', diet.column(1)>0)",
     'drink_or_not']]],
  1: [[['def drink(value):',
     '    if value == 0:',
     "        return 'False'",
     '    e